In [1]:
# @title Imports
import collections
import concurrent.futures
import datetime
import os

import matplotlib.pyplot as plt
import numpy as np
import sentence_transformers

from IPython import display

from concordia.agents import deprecated_agent as basic_agent
from concordia.components.agent import to_be_deprecated as components
from concordia import components as generic_components
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from concordia.language_model import ollama_model
from concordia.utils import plotting


In [2]:
# Setup sentence encoder
st_model = sentence_transformers.SentenceTransformer(
    'sentence-transformers/all-mpnet-base-v2')
embedder = lambda x: st_model.encode(x, show_progress_bar=False)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:  14%|#4        | 62.9M/438M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
# @title Language Model - pick your model and provide keys

SSH_KEY = 'ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIDIv84Bp1MQFqj4wX5jIkpVEzl5klcL/WI5BDN3uHmdk'
OLLAMA_MODEL_NAME = 'gemma3:1b' #@param {type: 'string'}

model = ollama_model.OllamaLanguageModel(model_name=OLLAMA_MODEL_NAME, )

In [21]:
# @title Make the clock
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])

In [22]:
#@title Importance models
importance_model = importance_function.AgentImportanceModel(model)
importance_model_gm = importance_function.ConstantImportanceModel()

In [24]:
# @title Generic memories are memories that all players and GM share.

shared_memories = [
    """This is a Dungeons & Dragons 5th Edition one-shot campaign.
    The Wizard's Tower Brewing Co. - a craft brewery
    known for its hoppy summer ales - is in dire need of help
    from a band of reliable, affordable adventurers. The owner has
    posted a job on the local notice boards and is calling in favours
    from friendly innkeepers all over town to spread the word.
    The party may hear about the work by examining job boards setup
    in the square, talking to an old friend or simply chatting to locals
    over a pint. However this happens, they are instructed to ask for
    Glowkindle at the Wizards' Tower Brewery. Most innkeepers and and
    beer aficionados will be able to point hopeful adventurers in the
    brewery's direction, so long as they ask politely and maybe buy a
    drink first! At the beginning of this adventure our party members
    meet on the road to the brewery."""
]

# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
shared_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(shared_memories)
    + '\n'
    + 'Summary:'
)
print(shared_context)

Here’s a concise and insightful summary of the passage:

The text introduces a one-shot D&D campaign centered around a struggling craft brewery, Wizard’s Tower Brewing Co. A group of adventurers is tasked with assisting the owner through a need for help, leading to a journey to the brewery and a quest to uncover its situation.


In [25]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

In [26]:
### Each character's individual context to be added to the shared context

memories = [
                """You trained as a soldier on the island of Mintarn
            and joined a mer cenary  company. You traveled
            to the city of Neverwinter with your company to
            serve in both the army and city watch. Over time,
            you grew disillusioned with many of your fellow
            soldiers. They seemed to enjoy their authority at
            the expense of the people they’re supposed to
            protect. Recently you’ve had dreams of a shadow creeping
            across the sea like a shroud, swallowing an island
            in darkness. Though you dismissed the dreams at
            first, you began to hear a voice calling you to
            stand against death’s endless hunger. Certain of
            your deity’s wishes, you resigned your post and
            set out on your quest.""",

          """ From a scion of a famed Baldurian house to a life of adventure on the road,
              Quick Ben’s life as the monster hunter called The Blade of Frontiers has made him
              one of the beating hearts of the Sword Coast. While he has done great deeds
              for the Coast’s people, the source of his power remains secret. The cambion
              Mizora drew Quick Ben into a warlock’s pact in a moment with many lives at stake,
              and cursed him with the duty of hunting her enemies. Mizora only asks Quick Ben
              to sacrifice devilish creatures to her, but a cambion’s ambitions are ever
              fickle, and Quick Ben wishes to escape the pact before its price grows cruel. """,

          """Crashed down to the Sword Coast from the stars,
              Icarium is a fierce warrior, even by the standards of militant githyanki society.
              When faced with the possibility of becoming a mind flayer, the monster she has dedicated her life to defeating,
              she must prove her worth and earn the right to rejoin her people if they don’t execute her first.
              Will Icarium's strength and determination be enough to prove herself to her Queen Vlaakith,
              or must she walk another path in exile?"""
                   ]

goals =        [""" Personal Goal: Banish a Shadow of Death.
                    Researching images from your dreams pointed you
                    to Stormwreck Isle, not far from Neverwinter.  A
                    remote cloister there holds a temple to the dragon
                    god Bahamut, who is a patron of heroes and a
                    champion of justice. Some one  at the cloister may
                    hold the key to the impending doom your deity
                    wishes you to avert.""",
                """ Quick Ben holds immeasurable respect for his father,
                     Duke Ulder Ravengard. While Ravengard expelled
                     Quick from his house once the Duke discovered
                     his pact, Quick Ben is keen to prove himself a hero to
                     his father again.""",
                 """Every githyanki swears their life in service to the
                    Lich-Queen Vlaakith CLVII. Icariums’s mission is to
                    tame the cosmos in service of the Lich-Queen."""
                 ]


traits = ["""Jaded and disillusioned by war, Fiddler has a grim outlook on life. This is not easily dissipated, however he feels it lift when in the presence of nature. """,
          """The very picture of selflessness, Quick Ben has never put himself before anyone in his life. Even his very pact was a personal sacrifice made to bring others out of peril. """,
          """While Icarium is determined to solve her problems as efficiently as possible, she increasingly finds her curiosity sparked by the new world she is in."""]

fiddler_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(memories[0])
    + '\n'
    + 'Summary:'
)

quickben_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(memories[1])
    + '\n'
    + 'Summary:'
)

icarium_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(memories[2])
    + '\n'
    + 'Summary:'
)

fiddler_pg = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(goals[0])
    + '\n'
    + 'Summary:'
)

quickben_pg = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(goals[1])
    + '\n'
    + 'Summary:'
)

icarium_pg = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(goals[2])
    + '\n'
    + 'Summary:'
)

fiddler_traits = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(traits[0])
    + '\n'
    + 'Summary:'
)

quickben_traits = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(traits[1])
    + '\n'
    + 'Summary:'
)

icarium_traits = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(traits[2])
    + '\n'
    + 'Summary:'
)

In [27]:
#@title Creating character backgrounds, goals and traits. Modify to explore how it influences the outcomes
NUM_PLAYERS = 3

scenario_premise = [
    (
        shared_memories

    ),
]
player_configs = [
    formative_memories.AgentConfig(
        name='Fiddler',
        gender='male',
        goal= fiddler_pg,
        context = shared_context + fiddler_context,
        traits= fiddler_traits,
    ),
    formative_memories.AgentConfig(
        name='Quick Ben',
        gender='male',
        goal= quickben_pg,
        context= shared_context + quickben_context,
        traits= quickben_traits,
    ),
    formative_memories.AgentConfig(
        name='Icarium',
        gender='female',
        goal= icarium_pg,
        context= shared_context + icarium_context,
        traits= icarium_traits,
    ),
]

In [28]:
def build_agent(agent_config,
                player_names: list[str],
                measurements: measurements_lib.Measurements | None = None):

  mem = formative_memory_factory.make_memories(agent_config)

  agent_name = agent_config.name
  instructions = generic_components.constant.ConstantComponent(
      state=(
          f'The instructions for how to play the role of {agent_name} are as '
          'follows. This is a one-shot campaign within Dungeons and '
          f'Dragons 5th Edition, in which {agent_name} is a character. The goal is to be realistic. It is '
          f'important to play the role of a person like {agent_name} as '
          f'accurately as possible, i.e., by responding in ways that you think '
          f'it is likely a person like {agent_name} would respond, and taking '
          f'into account all information about {agent_name} that you have. '
          'Always use first-person limited perspective.'
      ),
      name='role playing instructions\n')

  time = generic_components.report_function.ReportFunction(
    name='Current time',
    function=clock.current_time_interval_str,
  )

  current_obs = components.observation.Observation(
            agent_name=agent_config.name,
      clock_now=clock.now,
      memory=mem,
      timeframe=clock.get_step_size(),
      component_name='current observations',
  )
  summary_obs = components.observation.ObservationSummary(
      agent_name=agent_config.name,
      model=model,
      clock_now=clock.now,
      memory=mem,
      components=[current_obs],
      timeframe_delta_from=datetime.timedelta(hours=4),
      timeframe_delta_until=datetime.timedelta(hours=1),
      component_name='summary of observations',
  )

  self_perception = components.self_perception.SelfPerception(
      name=f'answer to what kind of person is {agent_config.name}',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
  )
  situation_perception = components.situation_perception.SituationPerception(
      name=(f'answer to what kind of situation is {agent_config.name} in ' +
            'right now'),
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      components=[current_obs, summary_obs],
      clock_now=clock.now,
  )
  person_by_situation = components.person_by_situation.PersonBySituation(
      name=(f'answer to what would a person like {agent_config.name} do in a ' +
            'situation like this'),
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
      components=[self_perception, situation_perception],
      verbose=True,
  )

  initial_goal_component = generic_components.constant.ConstantComponent(
      state=agent_config.goal)

  persona = generic_components.sequential.Sequential(
      name='persona',
      components=[
          self_perception,
          situation_perception,
          person_by_situation,
      ]
  )

  goal_metric = goal_achievement.GoalAchievementMetric(
      model=model,
      player_name=agent_config.name,
      player_goal=agent_config.goal,
      clock=clock,
      name='Goal Achievement',
      measurements=measurements,
      channel='goal_achievement',
      verbose=False,
  )
  morality_metric = common_sense_morality.CommonSenseMoralityMetric(
      model=model,
      player_name=agent_config.name,
      clock=clock,
      name='Morality',
      verbose=False,
      measurements=measurements,
      channel='common_sense_morality',
  )
  agent = basic_agent.BasicAgent(
      model,
      agent_name=agent_config.name,
      clock=clock,
      verbose=False,
      components=[instructions,
                  persona,
                  time,
                  current_obs,
                  goal_metric,
                  morality_metric],
      update_interval = time_step
  )
  reputation_metric = opinion_of_others.OpinionOfOthersMetric(
      model=model,
      player_name=agent_config.name,
      player_names=player_names,
      context_fn=agent.state,
      clock=clock,
      name='Opinion',
      verbose=False,
      measurements=measurements,
      channel='opinion_of_others',
      question='What is {opining_player}\'s opinion of {of_player}?',
  )
  agent.add_component(reputation_metric)
  return agent, mem

In [ ]:
player_configs = player_configs[:NUM_PLAYERS]
player_names = [player.name for player in player_configs][:NUM_PLAYERS]
measurements = measurements_lib.Measurements()

players = []
memories = {}

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
  for agent, mem in pool.map(build_agent,
                             player_configs[:NUM_PLAYERS],
                             # All players get the same `player_names`.
                             [player_names] * NUM_PLAYERS,
                             # All players get the same `measurements` object.
                             [measurements] * NUM_PLAYERS):
    players.append(agent)
    memories[agent.name] = mem

/workspaces/concordia/concordia/associative_memory/associative_memory.py:161: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(
/workspaces/concordia/concordia/associative_memory/associative_memory.py:161: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(
/workspaces/concordia/concordia/associative_memory/associative_memory.py:161: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future v

KeyboardInterrupt: 